In [1]:
import numpy as np
from bqplot import *
from bqplot.marks import Graph

In [2]:
from itertools import chain, product

class NeuralNet:
    def __init__(self, *args, **kwargs):
        self.height = kwargs.get('height', 600)
        self.width = kwargs.get('width', 960)
        self.layer_colors = kwargs.get('layer_colors', CATEGORY10)
        self.directed_links = kwargs.get('directed_links', False)
        
        self.nodes_input_layer = kwargs['nx']
        self.nodes_hidden_layers = kwargs['hidden_layers']
        self.nodes_output_layer = kwargs['ny']
        
        self.build_net()
    
    def build_net(self):
        # create nodes
        self.layer_nodes = []
        self.layer_nodes.append(['x' + str(i+1) for i in range(self.nodes_input_layer)])
        
        for i, h in enumerate(self.nodes_hidden_layers):
            self.layer_nodes.append(['h' + str(i+1) + ',' + str(j+1) for j in range(h)])
        self.layer_nodes.append(['y' + str(i+1) for i in range(self.nodes_output_layer)])
        
        self.flattened_layer_nodes = list(chain(*self.layer_nodes))
        
        # build link matrix
        i = 0
        node_indices = {}
        for layer in self.layer_nodes:
            for node in layer:
                node_indices[node] = i
                i += 1

        n = len(self.flattened_layer_nodes)
        self.link_matrix = np.empty((n,n))
        self.link_matrix[:] = np.nan

        for i in range(len(self.layer_nodes) - 1):
            curr_layer_nodes_indices = [node_indices[d] for d in self.layer_nodes[i]]
            next_layer_nodes = [node_indices[d] for d in self.layer_nodes[i+1]]
            for s, t in product(curr_layer_nodes_indices, next_layer_nodes):
                self.link_matrix[s, t] = 1
        
        # set node x locations
        self.nodes_x = np.repeat(np.linspace(0, 100, 
                                             len(self.layer_nodes) + 1, 
                                             endpoint=False)[1:], 
                                 [len(n) for n in self.layer_nodes])

        # set node y locations
        self.nodes_y = np.array([])
        for layer in self.layer_nodes:
            n = len(layer)
            ys = np.linspace(0, 100, n+1, endpoint=False)[1:]
            self.nodes_y = np.append(self.nodes_y, ys[::-1])
        
        # set node colors
        n_layers = len(self.layer_nodes)
        self.node_colors = np.repeat(np.array(self.layer_colors[:n_layers]), 
                                     [len(layer) for layer in self.layer_nodes]).tolist()
        
        xs = LinearScale(min=0, max=100)
        ys = LinearScale(min=0, max=100)
        
        self.graph = Graph(node_labels=self.flattened_layer_nodes, 
                           link_matrix=self.link_matrix, 
                           link_type='line',
                           colors=self.node_colors,
                           directed=self.directed_links,
                           scales={'x': xs, 'y': ys}, 
                           x=self.nodes_x, 
                           y=self.nodes_y)
        self.graph.hovered_style = {'opacity': '1', 
                                    'stroke': 'white'}
        
        self.graph.selected_style = {'opacity': '1',
                                     'stroke': 'white',
                                     'stroke-width': '2.5'}
        
        self.fig = Figure(marks=[self.graph])
        self.fig.layout.width = str(self.width) + 'px'
        self.fig.layout.height = str(self.height) + 'px'

In [3]:
nn = NeuralNet(nx=3, hidden_layers=[10, 5, 3], ny=1)
nn.fig

In [5]:
nn.graph.selected_style = {'opacity': 1.0, 'fill': 'DarkOrange', 'stroke': 'Red'}
nn.graph.unselected_style = {'opacity': 0.2}

In [6]:
def_tt = Tooltip(fields=['x', 'y'], formats=['', '.2f'])
nn.graph.tooltip = def_tt

In [7]:
from bqplot import pyplot as blt

In [8]:
fig_tt = blt.figure()
scat = blt.plot(np.cumsum(np.random.randn(100)))

In [9]:
nn.graph.tooltip = fig_tt